In [28]:
import numpy as np
import nltk, re, string, collections
import pandas as pd
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [29]:
with open("C:/Users/pc/Dataa.txt","r", encoding="utf-8") as file:
    data_text = file.read()
    print(data_text)
file.close()



        Дэлхийн хамгийн "царай муутай" баримлуудаас бүрдсэн лааны тосон музей
      
Бидний мэдэх алдартнуудын лааны тосон баримал үргэлж тэдэнтэй төстэй бодит юм шиг байна гэж боддог бол та эндүүрчээ. Та бидний төсөөлснөөс өөр нэгэн музей дэлхий ертөнц дээр оршин тогтнож байдаг. 


        Эрт үед бэлгийн замын өвчнийг хэрхэн эмчилдэг байсан бэ?
      
Хэдийгээр БЗХӨ эрт дээр үеэс улбаатай ч эмч нар ХХ зууны 2-р хагасаас эхлэн тус өвчнийг төгс эмчлэх болсон юм. Харин үүнээс өмнө БЗХӨ туссан хүмүүсийг маш хачирхалтай аргуудаар эмчилдэг байжээ. Гэхдээ тэдгээр нь хүмүүсийг эмчлэх гэхээсээ илүүтэй хорлож байсан гэхэд хилсдэхгүй юм. 


        Алдартнууд ямар гэмт хэрэгт холбогдож байсан бэ?
      
Алдартнууд бидний хувьд үлгэр дуурайлал, урам зоригийн бэлэг тэмдэг байдаг шүү дээ. Гэвч тэд ч мөн хүн л юм хойно элдэв асуудал, гэмт хэрэгт холбогддог байна. Ингээд танд гэмт хэрэг үйлдэж байсан 15 алдартныг танилцуулж байна.


        Жолоочгүй "Камаз" машиныг Хойд туйлд туршжээ
      
Оросы

In [30]:
data_text = re.sub('<.*>', '',data_text)
data_text = re.sub('ENDOFARTICLE.', '',data_text)
punctuationNoPeriod = "[" + re.sub("\.","",string.punctuation) + "]"
data_text = re.sub(punctuationNoPeriod, "", data_text)
print(data_text)



        Дэлхийн хамгийн царай муутай баримлуудаас бүрдсэн лааны тосон музей
      
Бидний мэдэх алдартнуудын лааны тосон баримал үргэлж тэдэнтэй төстэй бодит юм шиг байна гэж боддог бол та эндүүрчээ Та бидний төсөөлснөөс өөр нэгэн музей дэлхий ертөнц дээр оршин тогтнож байдаг 


        Эрт үед бэлгийн замын өвчнийг хэрхэн эмчилдэг байсан бэ
      
Хэдийгээр БЗХӨ эрт дээр үеэс улбаатай ч эмч нар ХХ зууны 2р хагасаас эхлэн тус өвчнийг төгс эмчлэх болсон юм Харин үүнээс өмнө БЗХӨ туссан хүмүүсийг маш хачирхалтай аргуудаар эмчилдэг байжээ Гэхдээ тэдгээр нь хүмүүсийг эмчлэх гэхээсээ илүүтэй хорлож байсан гэхэд хилсдэхгүй юм 


        Алдартнууд ямар гэмт хэрэгт холбогдож байсан бэ
      
Алдартнууд бидний хувьд үлгэр дуурайлал урам зоригийн бэлэг тэмдэг байдаг шүү дээ Гэвч тэд ч мөн хүн л юм хойно элдэв асуудал гэмт хэрэгт холбогддог байна Ингээд танд гэмт хэрэг үйлдэж байсан 15 алдартныг танилцуулж байна


        Жолоочгүй Камаз машиныг Хойд туйлд туршжээ
      
Оросын автомашин үйлдв

In [31]:
def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # токенүүдын дарааллыг сонгох
        seq = text[i-length:i+1]
        sequences.append(seq)
    print('Total Sequences: %d' % len(sequences))
    return sequences

# дарааллал бий болгох нийт дараалал 4730
sequences = create_seq(data_text)

Total Sequences: 4730


In [32]:
chars = sorted(list(set(data_text)))
mapping = dict((c, i) for i, c in enumerate(chars))

def encode_seq(seq):
    sequences = list()
    for line in seq:
        encoded_seq = [mapping[char] for char in line]
        sequences.append(encoded_seq)
    return sequences
sequences = encode_seq(sequences)
print(sequences)

[[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 21, 61, 46, 55, 43, 44, 48, 1, 55, 35, 47, 38, 43, 44, 48, 1, 56, 35, 51, 35, 44], [0, 1, 1, 1, 1, 1, 1, 1, 1, 21, 61, 46, 55, 43, 44, 48, 1, 55, 35, 47, 38, 43, 44, 48, 1, 56, 35, 51, 35, 44, 1], [1, 1, 1, 1, 1, 1, 1, 1, 21, 61, 46, 55, 43, 44, 48, 1, 55, 35, 47, 38, 43, 44, 48, 1, 56, 35, 51, 35, 44, 1, 47], [1, 1, 1, 1, 1, 1, 1, 21, 61, 46, 55, 43, 44, 48, 1, 55, 35, 47, 38, 43, 44, 48, 1, 56, 35, 51, 35, 44, 1, 47, 54], [1, 1, 1, 1, 1, 1, 21, 61, 46, 55, 43, 44, 48, 1, 55, 35, 47, 38, 43, 44, 48, 1, 56, 35, 51, 35, 44, 1, 47, 54, 54], [1, 1, 1, 1, 1, 21, 61, 46, 55, 43, 44, 48, 1, 55, 35, 47, 38, 43, 44, 48, 1, 56, 35, 51, 35, 44, 1, 47, 54, 54, 53], [1, 1, 1, 1, 21, 61, 46, 55, 43, 44, 48, 1, 55, 35, 47, 38, 43, 44, 48, 1, 56, 35, 51, 35, 44, 1, 47, 54, 54, 53, 35], [1, 1, 1, 21, 61, 46, 55, 43, 44, 48, 1, 55, 35, 47, 38, 43, 44, 48, 1, 56, 35, 51, 35, 44, 1, 47, 54, 54, 53, 35, 44], [1, 1, 21, 61, 46, 55, 43, 44, 48, 1, 55, 35, 47, 38, 43, 44, 48, 

In [33]:
from sklearn.model_selection import train_test_split
vocab = len(mapping)
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab)
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)

Train shape: (4257, 30) Val shape: (473, 30)


In [34]:
model = Sequential()
model.add(Embedding(vocab, 50, input_length=30, trainable=True))
model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
model.add(Dense(vocab, activation='softmax'))
print(model.summary())
model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
model.fit(X_tr, y_tr, epochs=100, verbose=2, validation_data=(X_val, y_val))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 50)            3350      
_________________________________________________________________
gru_1 (GRU)                  (None, 150)               90450     
_________________________________________________________________
dense_1 (Dense)              (None, 67)                10117     
Total params: 103,917
Trainable params: 103,917
Non-trainable params: 0
_________________________________________________________________
None


c:\users\pc\appdata\local\programs\python\python38\lib\site-packages\tensorflow\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


Train on 4257 samples, validate on 473 samples
Epoch 1/100
 - 3s - loss: 3.2968 - acc: 0.1882 - val_loss: 3.1491 - val_acc: 0.1903
Epoch 2/100
 - 3s - loss: 3.0172 - acc: 0.2067 - val_loss: 2.8970 - val_acc: 0.2558
Epoch 3/100
 - 3s - loss: 2.6955 - acc: 0.2624 - val_loss: 2.6421 - val_acc: 0.2939
Epoch 4/100
 - 3s - loss: 2.5235 - acc: 0.3009 - val_loss: 2.5535 - val_acc: 0.3362
Epoch 5/100
 - 3s - loss: 2.4102 - acc: 0.3324 - val_loss: 2.4956 - val_acc: 0.3531
Epoch 6/100
 - 3s - loss: 2.3174 - acc: 0.3580 - val_loss: 2.3971 - val_acc: 0.3615
Epoch 7/100
 - 3s - loss: 2.2202 - acc: 0.3798 - val_loss: 2.3331 - val_acc: 0.3848
Epoch 8/100
 - 3s - loss: 2.1527 - acc: 0.3925 - val_loss: 2.3106 - val_acc: 0.3784
Epoch 9/100
 - 3s - loss: 2.0828 - acc: 0.4101 - val_loss: 2.2894 - val_acc: 0.3827
Epoch 10/100
 - 3s - loss: 2.0273 - acc: 0.4186 - val_loss: 2.2444 - val_acc: 0.3827
Epoch 11/100
 - 3s - loss: 1.9778 - acc: 0.4322 - val_loss: 2.2232 - val_acc: 0.3890
Epoch 12/100
 - 3s - loss: 

Epoch 97/100
 - 2s - loss: 0.2142 - acc: 0.9363 - val_loss: 3.6718 - val_acc: 0.4186
Epoch 98/100
 - 3s - loss: 0.2266 - acc: 0.9316 - val_loss: 3.6690 - val_acc: 0.4292
Epoch 99/100
 - 2s - loss: 0.2226 - acc: 0.9345 - val_loss: 3.6882 - val_acc: 0.4292
Epoch 100/100
 - 3s - loss: 0.2232 - acc: 0.9274 - val_loss: 3.6954 - val_acc: 0.4355


In [36]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    for _ in range(n_chars):
        encoded = [mapping[char] for char in in_text]
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        yhat = model.predict_classes(encoded, verbose=0)
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        in_text += char
    return in_text

In [42]:
word = 'Хүн'
print(len(word))
print(generate_seq(model, mapping, 30, word.lower(), 15))

3
хүн ямар нэгэн хөг
